In [ ]:
import os
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir(os.pardir)

import deepxde as dde
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'text.usetex': True,
                     'text.latex.preamble': r'\usepackage{amsmath}',
                     'font.family': 'serif'})
import numpy as np
np.random.seed(0)
import pandas as pd

import seaborn as sns
sns.set(style='ticks', font='serif')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer

## Training data

In [ ]:
PATH_PROC_DATA = os.path.join('data', 'processed')
syn_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_synthetic.csv'))

features = ['d [mm]', 'f [GHz]', 'psPDtot_1 [W/m2]', 'psPDtot_4 [W/m2]']
target = 'pDeltaT * 100 [°C]'
X = syn_data[features]
y = syn_data[target] / 100

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y.to_numpy()[:, np.newaxis],
                                                    test_size=.2)

## Training the neural network

In [ ]:
data = dde.data.DataSet(X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test)

layer_size = [4] + [256] * 3 + [1]
activation = 'relu'
initializer = 'Glorot normal'
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)

optimizer = 'adam'
learning_rate = 1e-3
model.compile(optimizer=optimizer,
              lr=learning_rate,
              metrics=['l2 relative error'])
try:
    print('Trying to restore the neural network...')
    model.restore(os.path.join('models', '03_neural_network-500000.pt'))
    print('Restoring successful.')
    plot_loss = False
except Exception as e:
    print(e)
    print('Restoring failed. Training the neural network...')

    iterations = 500_000
    batch_size = 32
    display_every = int(iterations/10)
    start = timer()
    loss_history, train_state = model.train(iterations=iterations,
                                            batch_size=batch_size,
                                            display_every=display_every)
    end = timer()
    elapsed = start - end
    print(f'Training finished in {elapsed:.2f} s.')
    save_path = model.save(os.path.join('models', '03_neural_network'))
    print(f'Model saved ({save_path}).')
    plot_loss = True

In [ ]:
if plot_loss:
    cs = sns.color_palette('rocket', 3)
    dashes = [(3, 1), (1, 1)]

    fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))
    _x = np.arange(len(loss_history.loss_train)) * display_every
    ax.plot(_x, loss_history.loss_train, '-', c=cs[0], lw=2,
            label='train loss')
    ax.plot(_x, loss_history.loss_test, dashes=dashes[0], c=cs[1], lw=2,
            label='test loss')
    ax.plot(_x, loss_history.metrics_test, dashes=dashes[1], c=cs[2], lw=2,
            label='test metrics')
    ax.legend()
    ax.set(xlabel='', ylabel='',
           #xscale='log',
           yscale='log',
           xticks=[0, int(_x.max()/2), _x.max()],
           xticklabels=[0, int(_x.max()/2), _x.max()],
           # yticks=[],
           # yticklabels=[],
          )

    fig.supxlabel('iterations')
    fig.tight_layout()
    sns.despine()

    # fig_name = os.path.join('figures', '03_1_loss.png')
    # fig.savefig(fig_name, dpi=200, bbox_inches='tight')

## Evaluating the neural network

In [ ]:
true_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_clean.csv'))

X_eval = true_data[features]
y_eval = true_data[target].to_numpy() / 100

X_eval_scaled = scaler.transform(X_eval)

y_pred = model.predict(X_eval_scaled)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred.ravel()))

y_resid = (y_eval - y_pred.ravel())
ape = np.abs(y_resid) / y_eval
mape = np.mean(ape)

In [ ]:
cs = sns.color_palette('rocket', 2)

fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))
ax = sns.histplot(x=ape,
                  color=cs[0], stat='density', kde=True, ax=ax,
                  line_kws={'lw': 2})
ax.vlines(mape, *ax.get_ybound(), colors=cs[1], ls='--',
          label='mean absolute error')
ax.legend()
ax.set(xlabel='', ylabel='',
       xticks=[0, mape, int(ax.get_xbound()[1] / 2), int(ax.get_xbound()[1])],
       xticklabels=[0, round(mape, 2), int(ax.get_xbound()[1] / 2), int(ax.get_xbound()[1])],
       xlim=[0, int(ax.get_xbound()[1])],
       yticks=[0, 0.18, 0.36],
       yticklabels=[0, 0.18, 0.36],
       ylim=[0, 0.36]
      )

fig.supxlabel('absolute error [\%]')
fig.supylabel('probability density')
fig.tight_layout()
sns.despine()

# fig_name = os.path.join('figures', '03_2_ape_dist.png')
# fig.savefig(fig_name, dpi=200, bbox_inches='tight')